In [1]:
# Imports
import numpy as np
import optuna
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from hftbacktest import Recorder
from hftbacktest.stats import LinearAssetRecord

from src.strategies import glft

# Data Input paths
preprocessed_data_path = "../data/daily_processed"
daily_eod_snapshots = "../data/snapshots" # EOD = End Of Day 

C:\Users\codri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Taken from https://support.deribit.com/hc/en-us/articles/25944746248989-Fees
MAKER_FEE = 0   
TAKER_FEE =  0.0005    

In [3]:
from src.strategies.glft import gridtrading_glft_mm
day_start = 2
day_end = 22


def objective(trial):
    data = []
    latencies = []
    for i in range(day_start, day_end):
        day = str(i) if i > 9 else "0" + str(i)
        day_file = f"../data/daily_processed/deribit_eth_perp_2025-01-{day}.npz"
        day_latency = f"../data/latencies/latency_2025-01-{day}_latency.npz"
        data.append(day_file)
        # latencies.append(day_latency)
    day_start_str = str(day_start - 1) if day_start > 10 else "0" + str(day_start - 1)
    eod = np.load(f"../data/snapshots/deribit_eth_perp_2025-01-{day_start_str}_eod.npz")['data']

    gamma = trial.suggest_float("gamma", 0.01, 0.1)
    delta = trial.suggest_int("delta", 1, 10)
    adj1 = trial.suggest_float("adj1", 0.01, 2)
    adj2 = trial.suggest_float("adj2", 0.01, 1)
    max_position=50
    asset = (
    BacktestAsset()
        .data(data)
        .initial_snapshot(eod)
        .linear_asset(1.0)
        # .intp_order_latency(latencies, True)
        .constant_latency(10000, 10000) # Constant latency model (nanoseconds) values inspired from https://roq-trading.com/docs/blogs/2023-01-12/deribit/
        .risk_adverse_queue_model()
        # .power_prob_queue_model(2.0)
        .no_partial_fill_exchange()
        .trading_value_fee_model(MAKER_FEE, TAKER_FEE)
        .tick_size(0.01) # Tick size of this asset: minimum price increasement
        .lot_size(1) # Lot size of this asset: minimum trading unit
        # .roi_lb(0.0) # Sets the lower bound price for the range of interest in the market depth.
        # .roi_ub(3000.0) # Sets the upper bound price for the range of interest in the market depth.
        .last_trades_capacity(10000)
    )
    
    hbt = HashMapMarketDepthBacktest([asset])
  
    n_trading_days = day_end - day_start
    recorder = Recorder(1, n_trading_days* 1_000_000)
    
    gridtrading_glft_mm(hbt, recorder.recorder, n_trading_days, gamma, delta, adj1, adj2, max_position)
    hbt.close()

    stats = LinearAssetRecord(recorder.get(0)).stats()
    return stats.summary(pretty=True)['Return'][0]

In [4]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=42) # ensures reproducibility
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2025-05-24 21:52:39,934] A new study created in memory with name: no-name-2027d5b9-1e4f-46cc-b63c-0137c734aa21
[I 2025-05-24 21:57:56,553] Trial 0 finished with value: 141.0950000000007 and parameters: {'gamma': 0.04370861069626263, 'delta': 10, 'adj1': 1.4666679442046962, 'adj2': 0.6026718993550663}. Best is trial 0 with value: 141.0950000000007.
[I 2025-05-24 22:03:15,261] Trial 1 finished with value: -307791.6550000008 and parameters: {'gamma': 0.02404167763981929, 'delta': 2, 'adj1': 0.12558638821471693, 'adj2': 0.8675143843171859}. Best is trial 0 with value: 141.0950000000007.
[I 2025-05-24 22:09:02,731] Trial 2 finished with value: -98620.63499999975 and parameters: {'gamma': 0.0641003510568888, 'delta': 8, 'adj1': 0.05096314364864687, 'adj2': 0.9702107536403743}. Best is trial 0 with value: 141.0950000000007.
[I 2025-05-24 22:14:34,772] Trial 3 finished with value: -2661.5100000000634 and parameters: {'gamma': 0.08491983767203796, 'delta': 3, 'adj1': 0.37183168474213024, 'ad

In [5]:
study.best_params

{'gamma': 0.09623412767966626,
 'delta': 6,
 'adj1': 1.93155856130175,
 'adj2': 0.07016533681837103}